In [8]:
import psycopg2
import pandas as pd

sqluser = 'student'
dbname = 'mimic'
password = 'lipup'
host = 'luolabwkstn01.fsm.northwestern.edu'
port = '5432'
schema_name = 'mimiciii'

con = psycopg2.connect(dbname = dbname, user = sqluser, password = password, host = host, port = port)

In [9]:
query = """
SELECT
  pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.day,pvt.label
  
FROM
( -- begin query that extracts the data
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, ceiling((extract( epoch from ce.charttime - ie.intime))/60/60/24) as day
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
	     
  , CASE
      WHEN ITEMID = 444 THEN 'MeanAirwayPressure' 
      WHEN ITEMID = 1672 THEN 'MeanAirwayPressure'
      WHEN ITEMID = 224697 THEN 'MeanAirwayPressure'
      WHEN ITEMID = 470 THEN  'LPM'
      WHEN ITEMID = 471 THEN  'LPM'
      WHEN ITEMID = 223834 THEN 'LPM'
      WHEN ITEMID = 227582 THEN   'LPM'
      WHEN ITEMID = 64 THEN  'EPAP'
      WHEN ITEMID = 227579 THEN  'EPAP'
      WHEN ITEMID = 66  THEN  'IPAP'
      WHEN ITEMID = 227580  THEN  'IPAP'
      
      ELSE null
    END AS label , valuenum, value
FROM icu_18 ie

  LEFT JOIN mimiciii.chartevents ce
    ON ce.subject_id = ce.subject_id AND ce.hadm_id = ie.hadm_id
	
    AND ce.ITEMID in

    (
      444,1672,224697, -- cmH2O
	  470,471,223834,227582, -- LPM 
	  64,227579, -- EPAP
	  66,227580 -- IPAP
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
	where    ce.charttime BETWEEN (ie.intime) AND (ie.outtime)
) pvt
GROUP BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.day,pvt.label
ORDER BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id, day;
"""

df = pd.read_sql_query(query, con)
df

,subject_id,hadm_id,icustay_id,day,label
0,3,145834,211552,1.0,MeanAirwayPressure
1,3,145834,211552,2.0,MeanAirwayPressure
2,3,145834,211552,3.0,MeanAirwayPressure
3,3,145834,211552,4.0,LPM
4,3,145834,211552,4.0,MeanAirwayPressure
5,3,145834,211552,5.0,LPM
6,3,145834,211552,6.0,LPM
7,4,185777,294638,1.0,LPM
8,4,185777,294638,2.0,LPM
9,6,107064,228232,1.0,LPM


In [10]:
df.to_csv('interventions.csv')